In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

Macro `_latex_std_` created. To execute, type its name (without quotes).
=== Macro contents: ===
get_ipython().run_line_magic('run', 'Latex_macros.ipynb')
 

# Introduction


Up until now, the layers we have studed (Dense, Convolution) are primarily used to implement functions
that are one to one: 
- a single input (of fixed length) yields a single output.

Recurrent Neural Networks (RNN) deal with sequences: 
- sequences of inputs and sequence of outputs.

Sequences have order: a permutation of the elements of a sequence is a completely distinct sequence.

## Order matters !

Order matters
- set $\{ \x_{(1)} \dots \x_{(\tt-1)}\}$ versus sequence $\x_{(1)} \dots \x_{(\tt-1)}$
    - order doesn't matter for a set
    - $\{ \x_{(1)} \dots \x_{(\tt-1)}\} = \{  \x_{\tt-1)} \ldots \x_{(1)}\}$



<table>
    <tr>
        <th><center>Same prices</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_sequence_1.jpg" width=800></td>
    </tr>
</table>

<center>Same words</center>
$$
\begin{matrix}
\text{Machine} & \text{Learning} & \text{is} & \text{easy} & \text{not} & \text{difficult} \\
\text{Machine} & \text{Learning} & \text{is} & \text{difficult} & \text{not} & \text{easy} \\
\end{matrix}
$$


<table>
    <tr>
        <th><center>Same pixels</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_sequence_2.jpg" width=800></td>
    </tr>
</table>

All pairs of inputs in above examples are
- identical as sets
- different as sequences

**Note**

If the paired examples have different labels:
- NN will find as subset of features that separate them
- the separating feature then becomes an anchor, restricting reordering

## Functions on sequence

Examples of functions with sequence as inputs but single output (many to one mapping)
- predict next value in time series
- predict sentiment of text

Examples of functions with single input but sequence as output (one to many mapping)
- text generation (char-rnn) ?

Examples of functions with sequence as inputs and outputs
- translating from one language to another
- captioning a movie

**Notation alert**

Lot's of dimenions !
- $\x^\ip$ is a sequence with elements $\x^\ip_{(1)} \dots \x^\ip_{(\tt-1)}$
- each element $\x^\ip_\tp$ is a vector
    - $x^\ip_{(\tt),j}$ is a feature
        - examples
            - $\x^\ip_\tp$ is a frame $\tt$ in a movie; $x^\ip_{(\tt),j}$ is a pixel in frame $\tt$
            - $\x^\ip_\tp$ are the characteristics of a stock on day $\tt$, $x^\ip_{(\tt),j}$ is price or volume
            - $\x^\ip_\tp$ is a word $\tt$ in a sentence; $x^\ip_{(\tt),j}$ is element $j$ of the OHE of the word

Choices for how to predict $\y_\tp$ that is dependent on $\x_{(1)} \dots \x_\tp$

$$
\begin{array}[lll] \\
\pr{\y_\tp | \x_{(1)} \dots \x_\tp}  & \text{direct dependence on entire prefix } \x_{(1)} \dots \x_\tp \\
\\
\pr{\h_\tp | x_\tp, \h_{(\tt-1)} } & \text{latent variable } \h_\tp \text{encodes } \x_{(1)} \dots \x_\tp \\
\pr{\y_\tp | \h_\tp }              & \text{prediction contingent on latent variable} \\
\end{array}
$$

The Recurrent Neural Network (RNN) adopts the latter approach.

The single layer may also emit an output at step $i$ (for outputs that are sequences).

Here is some pseudo-code:

In [2]:
def RNN( input_sequence, state_size ):
    state = np.random.uniform(size=state_size)
    
    for input in input_sequence:
        # Consume one input, update the state
        out, state = f(input, state)
        
    return out
        

<table>
    <tr>
        <th><center>RNN</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_loop.jpg" width=1000></td>
    </tr>
</table>

Note that RNN's are sometimes drawn without separate outputs $\y_t$
- in that case, $\h_t$ may be considered the output. 

The computation of $\y_\tp$ will be just a linear transformation of $\h_t$ so there is no loss in omitting
it from the RNN and creating a separate node in the computation graph.

Geron does not distinguish betwee $\y_t$ and $\h_t$ and he uses the single $\y_\tp$ to denote the state.

I will use $\h$ rather than $\y$ to denote the "hidden state".


## $\h_\tp$ latent state

- $\h$ is a *fixed length* encoding of variable length sequence $\x_{(1)} \dots $
    - $\h_\tp$ encodes $\x_{(1)} \dots \x_\tp$
    - gives a way to have variable length input to, e.g., classifiers
- $\h_\tp$ is a vector of features
    - captures multiple "dimensions"/concepts of the input sequence


<table>
    <tr>
        <th><center>RNN Many to one; followed by classifier</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_many_to_one_to_classifier.jpg" width=800></td>
    </tr>
</table>

## Unrolling a loop
We can "unroll" the loop into the sequence of steps, with time as the horizontal axis

<table>
    <tr>
        <th><center>RNN unrolled</center></th>
    </tr>
    <tr>
        <td><img src="images/Unrolled_RNN.jpg" width=800></td>
    </tr>
</table>

Note that $\x, \y, \h$ are all vectors. 

In particular, the state $\h$ may have many elements
-  to record information about the entire prefix of the input.

The key connection is that the state at time $t-1$ is passed as input to time $t$.

So when processing $\x_\tp$
- the layer can take advantage of a summary ($\h_{(t-1)}$) of every input that preceded it.

One can look at this unrolled graph as being a dynamically-created computation graph.

Essentially, each state $\h$ is replicated per time step.

This view of the computation graph is important
- it shows you the exact computation
- it should tell you how gradients are computed
    - in particular, the loss and gradients flow backwards
        - so the gradients involving $\h$ are updated at *each* time step.
        - this will be important when we discuss
            - vanishing/exploding gradients
            - skip connections
            

# The RNN API

During one time step of computation, the RNN computes 2 values
- new  state $\h_\tp$
- output $\y_\tp$ (sometimes simply taken to be same as shor term state

The state computation is a function of the previous state $\h_{(t-1)}$,
and the current input $x_\tp$.

$$
\h_\tp = f(\x_\tp;  \h_{(t-1)})
$$

Note the recursive aspect of the computation of  $\h_\tp$: 
- it implicitly depends
on the values of the states at all previous time steps $t' < t$.


# RNN as a layer

## Many to one

Although the unrolled RNN looks confusing, as an "API" the RNN just acts as any other layer
- takes some input $\x$ (which happends to be a sequence)
- produces a single output

If we draw a box around the unrolled RNN, we can see the "API":

<table>
    <tr>
        <th><center>RNN many to one</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_many_one.jpg" width=800></td>
    </tr>
</table>


## RNN layer as an encoder
The many to one RNN essentially creates a compact encoding of an arbitrarily long sequence.

This can be very useful as we can feed this "summary" (representation) of the entire sequence
into layers that can't handle sequence inputs.

Note that there is nothing special about a layer creating a compact encoding (representation) of it's input.

A CNN layer, with outputs flattened to one dimension, creates a compact encoding of an image.

The real power of the RNN is the ability to encode all sequences, regardless of length, into a fixed size
representation.

### Sequences: variable length input summarized

$\h_\tp$ summarizes the length $\tt$ sequence $\x_{1,\ldots, \tt}$ in a *fixed size* vector $\h_\tp$.
- makes sequences amenable to models that can only deal with fixed size input


To be clear
- the RNN is a layer, just like any other
    - Internally it implements a loop but that is ordinarily hidden
    - The intuition about the "unrolled loop" is to help us to better understand the inner workings, not as a coding matter

- Like any other layer, it produces an output (although after multiple time steps for an RNN versus
a single time step for a Dense layer).
                                          
- If the length of sequence $\x$ is $T$, there is ordinarily a **single** output $\y_{(T)}$
    - $\y_{(T)}$ is only available after the entire input sequence has been consumed
    - the intermediate results 
    $$\h_\tp, \y_\tp, \; t = 1, \ldots, (T-1)$$ 
    are not visible through the API
    

## Many to many

The above behavior defines a many to one mapping from input sequence (many) to single output (one).

With a minor change, we can define a many to many mapping:
- each element of the input sequence
results in one element of an output sequence.

Many Deep Learning software API's will see recurrent layers with an optional
argument 
- `return_sequences`
- `return_states` 
- both default to `False` in Keras.

This controls the output behavior of the RNN layer, whether it returns one output per time step
$$
       \h_{(1)}, \ldots, \h_{(T)} \\
       \y_{(1)}, \ldots, \y_{(T)}
$$
or just
$$
\h_{(T)} \\
\y_{(T)}
$$

This is how any RNN behaves when the function it's implementing is many to many:
- one output per time step.

When the RNN needs to implement a many to one function, the layer looks like

<table>
    <tr>
        <th><center>RNN many to many</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_many_many.jpg" width=800></td>
    </tr>
</table>


The art-work needs to be clarified
- the RNN layer produces sequences
    - as outputs $\y$
    - as states $\h$

These sequences are available when the RNN layer *completes* its consumption of input $\x$.

The following diagram may clarify


<table>
    <tr>
        <th><center>RNN many to many, clarified</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_loop_many_many.jpg" width=1000></td>
    </tr>
</table>

- the `return_sequences` argument instructs the layer to produce a sequence $\y$
    - rather than a scalar, as in the many to one case
- the `return_states` argument instructs the layer to return the state $\h$ as well
    - useful if we stack RNN layers

### Stacked RNN layers

One can connect RNN layers into "stacks" 
- by feeding
the output state of one RNN layer as the input to the successor layer:


<table>
    <tr>
        <th><center>RNN Stacked layers</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layers_stacked.jpg" width=800></td>
    </tr>
</table>
​

### Encoder/Decoder architecture

An *Encoder/Decoder* is a two part Neural Network that is applied to many NLP tasks
- *Encoder* converts sequence (sentence) into intermediate representation (sequence)
- *Decoder* converts intermediate sequence to final sequence


<table>
    <tr>
        <th><center>RNN Encoder/Decoder</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_Encoder_Decoder.jpg" width=800></td>
    </tr>
</table>
​


### Sequence to Sequence

- Many to one encoder
    - variable length input to fixed length final state $\h$
- One to one decoder
    - *initial* state of decoder set to *final* state of encoder
    - teacher forcing
        - input $(\tt +1)$ of decoder is out $\tt$ of decoder

Example: language translation

This is useful when there is not an exact one to one correspondence between tokens in the source and target languages.

# Training: forward/backward pass, cost/loss

Examples
- an example $\x^\ip$ is now a *sequence* $\x^\ip_{(1)}, \x^\ip_{(2)}, \ldots, \x^\ip_{(T)} $
    - variable length
    - $\x^\ip_\tp$ *may* be a vector (doesn't have to be scalar), 
        - e.g., word embedding
        

Per example loss $\loss^\ip$ *per time step*
- In many to many: there is a loss per time-step.  
- Total loss (over which we optimize) is sum, orver time , of the loss per time step
    - $\loss^\ip = \sum_{\tt=1}^n \loss^\ip_\tp$
- In many to one: loss is single value (per example): depends on final state 
    - $\loss^\ip = \loss_{(T)}$ 

# RNN details: update equations

$$
\begin{array}[lll]\\
\h_\tp & = & \phi(\W_{xh}\x_\tp  + \W_{hh}\h_{(t-1)}  + \b_h) \\
\y_\tp & = &  \W_{hy} \h_\tp  + \b_y \\
\end{array}
$$

where $\phi$ is an activation function (usually $\tanh$)

**Note** Geron prefers right multiplying weights $\x_\tp \W_{xh}$ versus $\W_{xh}\x_\tp$
- left multiplying seems more common in literature

**Note**
The equation is for a single example.  

In practice, we do an entire minibatch so have $m$ $\x's$
given as a $(m \times n)$ matrix $\mathbf{X}$.

**page 471: mention dimensions of each**

## Equation in pseudo-matrix form

You will often see a short-hand form of the equation.

Look at $\h_\tp$ as a function of two inputs $\x_, \h_{(t-1)}$.

We can stack the two inputs into a single matrix.

Stack the two matrices $\W_{xh}, \W_{hh}$ into a single weight matrix

$
\begin{array}[lll]\\
\h_\tp  = \W \mathbf{I} + \b \\
\text{ with } \\
\W = \left[
 \begin{matrix}
    \W_{xh} & \W_{hh}
 \end{matrix} 
 \right] \\
\mathbf{I} = \left[
 \begin{matrix}
    \x_\tp  \\
    \h_{(t-1)}
 \end{matrix} 
 \right] \\
\end{array}
$

## Stacked RNN layers revisited

With the benefit of the RNN update equations, we can clarify how stack RNN layers works>

Let superscript $[\ll]$ denote a stacked layer of RNN.

So the RNN update equation for the bottom layer $1$ becomes
$$
\begin{array}[lll]\\
\h^{[1]}_\tp & = & \phi(\W_{xh}\x_\tp  + \W_{hh}\h^{[1]}_{(t-1)}  + \b_h) \\
\end{array}
$$

The RNN update equation for leyer $[\ll]$ becomes

$$
\begin{array}[lll]\\
\h^{[\ll]}_\tp & = & \phi(\W_{xh}\h^{[\ll-1]}_\tp  + \W_{hh}\h^{[\ll]}_{(t-1)}  + \b_h) \\
\end{array}
$$

That is: the input to layer $[\ll]$ is $\h^{[\ll-1]}_\tp$ rather than $\x_\tp$

# Back propagation through time (BPTT)

<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>We can "unroll" the RNN into a sequence of layers, one per time step</li>
        <li>In theory: Back Propagation on the unrolled RNN is the same as for a non-Recurrent Network</li>
        <li>In practice: the unrolled RNN is very deep, which causes issues in Back Propagation.
    </ul>
</div>


*Back Propagation Through Time (BPTT)* refers to
- unrolling the RNN computation into a sequence of layers
- performing ordinary Back Propagation in order to update weights

- In a non-Recurrent network:
    - $\W_\llp$, the weights of layer $\ll$, affect only layers $\ll$ and greater.
    - This means the backward flow of the gradient with respect to $\W_\llp$ stops at layer $\ll$.
- In  Recurrent Network:
    - All unrolled "layers" share the *same* weights
    - This means the gradients with respect to shared weight $\W$ must flow backward all the way to the input layer at time $0$.
    

<table>
    <tr>
        <th><center>RNN Loss Gradient Flow</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_loss_gradient.jpg" width=800></td>
    </tr>
</table>

The unrolled graph is as deep as the length of $\x^\ip$ ($T^\ip = |\x^\ip|)$ 
- weights can update only after $T^\ip$ input values have been processed,
so training can be slow.
- Vanishing Gradients become a concern for large $T^\ip$
    - Recall from the Vanishing Gradient lecture: magnitude of gradients diminishes from layer $\ll$ to layer $(\ll-1)$ during back propagation

## Calculating gradients with BPTT

### Back propagation: Refresher

The same math that we used to show how to obtain derivatives (for weight updates in Gradient Descent)
will apply to RNN's.

To refresh our memory on notation and results, recall our derivation of back propagation:

Layer $\ll$:
- input/output relation of layer $\ll$ as
$$
\y_{(\ll)} = a_{(\ll)}( f_{(\ll)}( \y_{(\ll-1)}, \W_{(\ll)}) )
$$

for
- activation function $a_\llp$
- weights $\W_\llp$
- $\y_{(\ll-1)}$ are the outputs of the previous layer

- $f_{(\ll)}$ is the function computed by layer $\ll$
    - function of input $\y_{(\ll-1)}$ and weights $\W_\llp$
    - e.g., `Dense`: $f_{(\ll)}( \y_{(\ll-1)}, \W_{(\ll)}) = \y_\llp = \W_\llp \y_{(\ll-1)} + \b_\llp$

**Note** We neglect to add $\b_\llp$ as an argument to $f_\llp$ to simplify notation
- as a convenience we sometimes view $\b_\llp$ as being part of $\W_\llp$

Let 
- $\loss$ denote loss (computed after final layer $L$)
- $\loss'_\llp = \frac{\partial \loss}{\partial y_\llp}$ denote the derivative of $\loss$ with respect to the output of layer $\ll$, i.e., $y_\llp$,
    - refer to as **loss gradient** (at output of layer $\ll$)
    

We showed how to compute
- $\loss'_{(\ll-1)}$ from $\loss'_\ll$ 
    - so that we can continue this process as the previous layer (i.e, *propogate loss gradient backwards*)

and we showed how to compute the weight update
- $\frac{\partial \loss}{\partial W_\llp}$, from $\loss'_\llp$  for $\ll \in [1,L]$

Note that $\y_\llp$ is a function of 
- $\y_{(\ll-1)}$ (the output of the previous layer) 
- and $\W_\llp$, the parameters of layer $\ll$.

We can compute derivatives of $\y_\llp$ with respect to each of its inputs
- $\frac{\partial \y_\llp}{\partial \y_{(\ll-1)}}$
- $\frac{\partial \y_\llp}{\partial \W_\llp}$

Refer to these as **local gradients**

We used the chain rule to obtain the 
- gradient with respect to weights $\W_\llp$, given the loss gradient $\loss'_\llp$ 

$$
\begin{array}[lll] \\
\frac{\partial \loss}{\partial \W_\llp} & = & \frac{\partial \loss}{\partial \y_\llp} \frac{\partial \y_\llp}{\partial \W_\llp} & = & \loss'_\llp \frac{\partial \y_\llp}{\partial \W_\llp}
\end{array}
$$

That is: 
- gradient of $\loss$ with respect to weight $\W_\llp$ 
- is the loss gradient (at current step), multiplied by
- a local gradient (with respect to input  $W_\llp$ )

So we have the information required to update $\W_\llp$ by Gradient Descent.


### BPTT: gradient calculation

Let us adapt these results for the case of a *single layer* RNN
- by "unrolling" this RNN, layer $\ll$ is equated with "time" (of index into input sequence ) $\tt$

Per example loss $\loss^\ip$ is now a per example loss *per time step*
$$
\loss^\ip_\tp
$$
so
$$
\loss^\ip = \sum_{t=1}^T \loss^\ip_\tp
$$

We will focus on the per example loss for a single time $\loss^\ip_\tp$

​
<table>
    <tr>
        <th><center>RNN Loss</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_loss.jpg" width=800></td>
    </tr>
</table>

In order to compute 
$$
\frac{\partial \loss^\ip}{\partial \W}
$$
we must compute
$$
\frac{\partial \loss^\ip_\tp}{\partial \W}
$$
for each time $t$.

As per regular backprop, we can obtain the loss update by
multiplying the loss gradient by a local gradient

$$
\frac{\partial \loss^\ip_\tp}{\partial \W}
$$

but note that we use unsubscripted $\W$ (rather than $\W_\tp$ because the *same* $\W$ is used at all timesteps.

$$
\frac{\partial \loss^\ip_\tp}{\partial \W} = \loss'_\tp \frac{\partial \y_\tp}{\partial W}
$$

but now 
$$
\frac{\partial \y_\tp}{\partial W}
$$

becomes more complicated, governed by the RNN Update equations

$$
\begin{array}[lll]\\
\h_\tp & = & \phi(\W_{xh}\x_\tp  + \W_{hh}\h_{(t-1)}  + \b_h) \\
\y_\tp & = &  \W_{hy} \h_\tp  + \b_y \\
\end{array}
$$

**Notes**

- In this section we will assume $\phi$ is the identity function to simplify the presentation.

    - There will be no loss of generality.
- Recall that $\W$ is the matrix with embedded sub-matrices $\W_{xh}, \W_{hh}, \W_{hy}$
    - For clarity: we will add subscripts to $\W$ in the derivatives to show which part of $\W$ is the cause.

The equation defining $\y_\tp$
$$
\y_\tp  =   \W_{hy} \h_\tp  + \b_y 
$$

shows that $\y_\tp$ is
- directly depends on  $\W$ (through $\W_{hy}$ )
- *and* indirectly depends on $\W$  through its dependence on $h_\tp$ (which depends on $\W$)

So
$$
\frac{\partial \y^\ip_\tp}{\partial \W} = 
\frac{\partial \y^\ip_\tp}{\partial \W_{hy}}
+
\frac{\partial \y^\ip_\tp}{\partial \h_{(\tt)} } \frac{\partial \h_{(\tt)}}{\partial \W_{hh}}
$$

Let's expand the term
$$
\frac{\partial \h_{(\tt)}}{\partial \W_{hh}}
$$

Recall the recursive definition of $\h_\tp$ 
$$
\h_\tp =  \W_{xh}\x_\tp  + \W_{hh}\h_{(\tt-1)}  + \b_h
$$

$\h_\tp$ depends on $\h_{(\tt-1)}$, which by recursion depends on $\h_{(\tt-2)}$ which $\ldots$ depends on $\h_{(0)}$.
- and all $\h_\tp$ share the *same* $\W_{hh}$.

This means that $\h_\tp$ depends on $\W$ through *each* $\h_{(\tt-k)}$ for $k=1, \ldots, \tt$.
$$
\frac{\partial \h_\tp}{\partial \W_{hh}} = 
\sum_{k=1}^{\tt} { 
\frac{\partial \h_{(\tt -k)}}{\partial \W_{hh}} \frac{\partial \h_\tp}{\partial \h_{(\tt -k)}}      
}
$$

So 

$$
\frac{\partial \loss^\ip_\tp}{\partial \W} = \loss'_\tp \frac{\partial \y_\tp}{\partial W}
$$

and
$$\frac{\partial \y^\ip_\tp}{\partial W}$$
*depends* on all time steps from $1$ to $t$.

Thus, the derivative update for $\W$ cannot be computed without the gradient (for each time step $t$)
flowing all the way back to time step $0$.


**Note**

Directly expanding the recursion would show
$$
\frac{\partial \h_\tp}{\partial \h_{(\tt -k)}} = 
\prod_{k'=0}^{k-1} { 
\frac{\partial \h_{(\tt-k')}}{\partial \h_{(\tt -k' -1)}}
} 
$$

It is not necessary now, but will be useful in explaining vanishing/exploding gradients

<table>
    <tr>
        <th><center>RNN Loss Gradient Flow</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_loss_gradient.jpg" width=800></td>
    </tr>
</table>

## Truncated back progagation through time (TBTT)

<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>We "unroll" the RNN into a sequence of  T layers, one per time step</li>
        <li>We compute the loss at each time step t, for t=1 to T.</li>
        <li>The gradient of the loss of time step t flows backward for a limited number of time steps</li>
        <ul>
            <li>Rather than flowing backwards al the way to time step 0</li>
        </ul>
    </ul>
</div>

This is called *Truncated BPTT (TBPTT)*.

The advantage of TBPTT
- more frequent gradient updates

The disadvantage
- the loss at step $t$ won't affect **all** previous time steps (because of truncation)
- the error signal from time $\tt$ does not affect any time steps below $\tt - \tau$.
- this means the RNN has difficulty capturing dependencies longer than $\tau$.

Consider a long piece of text
- The first few words indicate the gender/plurality/age of the subject
- A mis-prediction of, e.g. gender, at word $\tau' > \tau$ causes an error at time step $\tau'$
    - which can't interact with the correct gender in the first few words


Note that there is *no truncation* of the forward pass of the RNN !

Only gradient calculations are truncated.

### TBTT: Variations

There are several ways to truncate the Back Propagation.

We will describe them via a function $f(\tt) = \tt'$
- describes the earliest time
step affecting the gradient of $\loss_\tp$
- that is, it describes the window $\tau$

- Untruncated BPTT
    - $f(\tt) = 0$
- k-truncated BPTT
    - $f(\tt) = \max{}(0, \tt -k)$
- subsequence truncated BPTT
    - $f(\tt) = k * \floor{\tt/k}$

What we refer to as subsequence TBTT seems to be common
- break long sequence $\x^\ip$ into subsequences (chunks) of size $k$
- feed $\x^\ip$ forward as usual
    - at the end of a subsequence: 
        - immediately compute the loss gradients for all time steps within the chunk


# RNN vanishing/exploding gradient problem


<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>A "single-layer RNN that has been unrolled for T time steps</li>
        <ul>
            <li>is mathematically equivalent to a simple NN with T layers</li>
            <li><bold>BUT</bold> all layers share the same weights</li>
        </ul>
        <li>This sharing of weights leads to a problem of Vanishing/Exploding gradients</li>
        <ul>
            <li>Similar to the vanishing gradient problem we derived for simple NN</li>
            <li>but with a different root cause (weight sharing)</li>
        </ul>
    </ul>
</div>

<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>Why shared weights are different</li>
        <ul>
            <li>Output <bold>y</bold> at time t is a function of cell state h at time t</li>
            <li>Cell state h at time t is recursively defined</li>
            <ul>
                <li>So it is a function of cell states over all times t' < t as well</li>
                <li>This means the weight update involves a repeated product: (t -t') times</li>
                <li>This product tends to 0 (vanishing) or infinity (explode) as (t -t') increases</li>
            </ul>
            <li>So losses at time step t have difficulty updating gradients for the distant past<</li>
            <li>RNN has difficulty with long-term dependencies</li>   
        </ul>
    </ul>
</div>

Returning to the loss gradient
we encountered the terms

$$\frac{\partial \y^\ip_\tp}{\partial \W}$$

We will focus on the part of $\W$ that is $\W_{hh}$

$$
\frac{\partial \y_\tp}{\partial W_{hh}} = \frac{\partial \y_\tp}{\partial \h_\tp} \frac{{\partial \h_\tp}}{\partial \W_{hh}}
$$

But recursively defined $\h_\tp$ is a function of $\h_{(\tt-1)}, \h_{(\tt-1)}, \ldots, \h_{(1)}$
so

$$
\frac{\partial \y_\tp}{\partial W_{hh}} = \frac{\partial \y_\tp}{\partial \h_\tp}
 \sum_{k=0}^\tt {  \frac{\partial \h_\tp}{\partial \h_{(\tt -k)}} \frac{\partial \h_{(\tt -k)}}{\partial \W_{hh}} }
$$

The summation: $\frac{\partial \h_\tp}{\partial \W_{hh}}$, through all intermediate $\h_{(\tt -k)}$


The problematic term for us is
$$
\frac{\partial \h_\tp}{\partial \h_{(\tt-k)}}
$$

It can be computed by the Chain Rule as
$$
\frac{\partial \h_\tp}{\partial \h_{(\tt-k)}} = \prod_{u=0}^{\tt-1} { \frac{\partial \h_{(\tt -u)}}{\partial \h_{(\tt - u -1) }}}
$$

Each term
$$
\frac{\partial \h_{(\tt -u)}}{\partial \h_{(\tt - u-1)}}
$$
results in a term $\W_{hh}$ so the repeated product compute matrix $\W_{hh}$ raised to the power $k$.



For simplicity,  suppose $\W_{hh}$ were a scalar
- if $\W_{hh} < 1$ then repeatedly multiply $\W_{hh}$ by itself approaches $0$
- if $\W_{hh} > 1$ then repeatedly multiply $\W_{hh}$ by itself approaches $\infty$

In other words:
- as the distance between time steps $\tt$ and $(\tt -k)$ increases
- the gradient (for the weight  update) either vanishes or explodes.

Since this term is used in the update for our weights
- updates  will either be erratic (too big) 
- or
non-existent, hampering learning of weights.

This was not necessarily a problem in non-recurrent networks 
- because each layer had a different
weight matrix.

What an RNN does that helps it be parsimonius in number of parameters
- by sharing the weights across
all time steps
- hurts us in learning.
                                              

For the general case where $\W_{hh}$ is a matrix
- we can show the same resul with the eigenvalues of the matrix

### Controlling exploding gradients by clipping
In theory, we can control the explosion by clipping the gradient $\frac{\partial \loss}{\partial W_i}$.

We are still left with the vanishing gradient problem.

This means that we can't learn long-term dependencies (i.e., too many steps backward).

This will be "solved" by introducing recurrent architectures that address this issue.

# Long sequences in Tensorflow/Keras

Dealing with something as "simple" as sequences can be surprisingly difficult in Tensorflow/Keras.
- One is required to manually break up long sequences into multiple, shorter subsequences
- The ordering of the examples in a mini-batch now becomes relevant


Consider a long sequence $\x^\ip$ of length $n$.

The "natural" way to represent this $\X$ is

$$
\X = 
\begin{pmatrix}
\x^{(1)}_{(1)} & \x^{(1)}_{(2)} & \ldots & \x^{(1)}_{(n^{(1)})} \\
\x^{(2)}_{(1)} & \x^{(2)}_{(2)} & \ldots & \x^{(2)}_{(n^{(2)})} \\
\vdots
\end{pmatrix}
$$

for equal example lengths $n^{(1)} = n^{(2)} \ldots$

Suppose that the example lengths $n$ is too big.

That is, they areto be broken up into subsequences of length $n'$.

There will be $n/n'$ such subsequences.

We write $\x^{(i, \alpha)}$ to denote subsequence number $\alpha$ in examples $i$.
- The elements of this subsequence are $\x^{(i,\alpha)}_\tp$ for $1 \le \tt \le n'$.
- So $\x^{(i,\alpha +1)}_{(1)}$ follows $\x^{(i,\alpha)}_{(n')}$

TensorFlow (as of the time of this writing) has limited primitive concepts
- examples *within* batches are unrelated
- example $i$ of one batch *can* be made to be related to example $i$ of the following batch
    - optional flag

To get adjacent subsequences of one sequence to be treated in the proper order by TensorFlow:
- Define the number of minibatches to be $n/n'$, which is the number of subsequences
- Each subsequence of example $i$ should be at the *same position* within each of the $n/n'$ minibatches
- Set RNN optional parameter `stateful=True`
- When fitting the model: set `shuffle=False`

$$
\text{Minibatch 1} = 
\begin{pmatrix}
\x^{(1)}_{(1)} & \x^{(1)}_{(2)} & \ldots & \x^{(1)}_{(n')} \\
\x^{(2)}_{(1)} & \x^{(2)}_{(2)} & \ldots & \x^{(2)}_{(n')} \\
\vdots
\end{pmatrix}
$$


$$
\text{Minibatch 2} = 
\begin{pmatrix}
\x^{(1)}_{(n' +1)} & \x^{(1)}_{(n' +2)} & \ldots & \x^{(1)}_{(n' +n')} \\
\x^{(2)}_{(n' +1)} & \x^{(2)}_{(n' +2)} & \ldots & \x^{(2)}_{(n' +n')} \\
\vdots
\end{pmatrix}
$$

`stateful=True`
- TensorFlow *will not** reset the state of the RNN at the beginning of a new minibatch (for each example in the minibatch)

This means that 
- the example at position $i'$ within each minibatch share state ($\h$)
- we've ordered the minibatches in the correct ordering of subsequences
- so the result is that the entirety of example $i$'s time steps update the same state

The main difference from the simple method (one example with a long sequence)
- we've broken the example up into subsequences
- that are related by common position within adjacent minibatches
    -

`shuffle=False`
- Examples in adjacent minibatches must be in the *same* order, so don't shuffle them !

# Sequences: Variable length

There are lots of small potholes one encounters with sequences.

What is the examples of my training set have widely varying lengths ?

- Within a batch, short examples may behave differently than long examples:
    - Maybe learn less in short examples, noisier gradient updates
    
- Padding sequences to make them equal length
    - Pad at the start ? Or at the end ?

The general advice is to arrange your data so that an epoch contains examples of similar lengths.
- You may require multiple fittings, one per length

# Residual connections: a gradient highway
[Deep Residual Learning](https://arxiv.org/abs/1512.03385)

<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>We have encountered the Vanishing Gradient problem several times.</li>
        <li>This is a major impediment to training deep (many layers) networks.</li>
        <li>The solution is to give the gradient a path to flow backward undiminished.</li>
        <li>This simple solution is called a Skip or Residual Connection</li>
    </ul>
</div>

Consider two layers of a NN
$$
\begin{array}[lll]\\
\y_{(\ll-1)}      & = & a_{(\ll-1)}      & \left( f_{(\ll-1)}( \y_{(\ll-2)} ) \right) \\
\y_{\llp} & = & a_\llp & \left( f_\llp ( \y_{(\ll-1)} ) \right) \\
\end{array}
$$

Suppose we modify layer $\ll +1$ by adding $\y_{(\ll-1)}$ to the  output

$$
\begin{array}[lll]\\
\y_{(\ll-1)}      & = & a_{(\ll-1)}      & \left( f_{(\ll-1)}( \y_{(\ll-2)} ) \right) \\
\y_{\llp} & = & a_\llp & \left( f'_\llp ( \y_{(\ll-1)} ) \right) + \y_{(\ll-1)}   \\
\end{array}
$$


If the original and modified 2 layer mini-networks compute the same function from $\y_{(\ll-1)}$ to $\y_{(\ll+1)}$

$$
\begin{array}[lll]\\
a_\llp \left( f_\llp( \y_{(\ll-1)} ) \right)   & = & a_\llp  \left( f'_\llp ( \y_{(\ll-1)}) \right)  + \y_{(\ll-1)} \\
a_\llp \left( f'_\llp ( \y_{(\ll-1)} ) \right) & = & a_\llp \left( f_\llp( \y_{(\ll-1)} ) \right) - \y_{(\ll-1)}  \\
\end{array}
$$

In other words: 
- we have forced the modified second layer to learn the "residual" of the unmodified layer with respect to $\y_{(\ll-1)}$.

This seems strange (and pointless) until you consider the Back Propagation process.

Recall how the loss gradient
$$\loss'_\llp = \frac{\partial \loss}{\partial y_\llp}$$

propagates backwards
$$
$$
\begin{array}[lll] \\
\loss'_{(\ll-1)} 
         & = & \loss'_\llp \frac{\partial \y_\llp}{\partial \y_{(\ll-1)}}
\end{array}
$$
$$
   

In the unmodified mini-NN  the local derivative
$$
\frac{\partial \y_\llp}{\partial \y_{(\ll-1)}} = \frac{ a_\llp \left(f_\llp(\ldots) \right) }{\partial \y_{(\ll-1)}} 
$$

whereas, in the modified mini-NN the local derivative becomes
$$
\frac{\partial \y_\llp}{\partial \y_{(\ll-1)}} = \frac{ a_\llp \left( f'_\llp (\ldots)\right) }{\partial \y_{(\ll-1)}} + 1
$$

When $\frac{\partial \y_\llp}{\partial \y_{(\ll-1)}}$ is multiplied by $\loss'_\llp$
to obtain $\loss'_{(\ll-1)}$
- the "upstream" loss gradient $\loss'_{(\ll-1)}$
- flows backwards to layer $\ll -1$ unmodulate *because of the* $+ 1$ term in the modified local derivative.

This simple trick vanquishes the vanishing gradient !

It is one of the major reasons that we are able to train extremely deep NN's.

There is another important implication:
- adding an additional layer cannot result in increased loss

This is because there exists a set of weights $\W_\llp$ for which 
$$
a_\llp \left( f'_\llp ( \y_{(\ll-1)} ) \right) = 0
$$

This means the modified second layer computes the identity functon
$$
\y_\llp = \y_{(\ll-1)}
$$

So if adding the second layer had the potential of increasing loss relative to a one layer network
- the optimization would learn the identity function instead, an no increase in loss rsults-

Without the skip connection, it is empircally difficult for a NN to learn an identity function as a layer.

There is an unresolved debate where to place the "head" of the skip connection
- insider the activation function
- outside the activation function

We choose the latter to simplify the derivative expression for the loss gradient.

<table>
    <tr>
        <th><center>"Plain" Neural Network</center></th>
    </tr>
    <tr>
        <td><img src="images/ResNet_plain.jpg" width=800"></td>
    </tr>
</table>

<table>
    <tr>
        <th><center>Neural Network with skip connection</center></th>
    </tr>
    <tr>
        <td><img src="images/ResNet_skip.jpg" width=800"></td>
    </tr>
</table>

## Preview: skip connections in LSTM's, GRU's

The gradient highway also turns out to be useful in RNN's.

There are more powerful variants of the RNN called LSTM and GRU which avoid vanishing gradients, partially 
through the use of skip connections.

These variants enhance the power of skip connections by allowing selective skipping via the use
of "gates".

We will see this shortly.

# Visualization of RNN hidden state

Here is a [visualization](http://karpathy.github.io/2015/05/21/rnn-effectiveness/#visualizing-the-predictions-and-the-neuron-firings-in-the-rnn) of single elements within the hidden state, as they consume the input sequence
of *single characters*.

The color reflects the intensity (value) of the paricular cell (blue=low, red=high)


<table>
    <tr>
        <th><center>State activations after seeing prefix of input</center></th>
    </tr>
    <tr>
        <td><img src="images/Unreasonable_effectiveness_1.png" width=800></td>
    </tr>
</table>

# RNN as a generative model


Up to now, an RNN's inputs were a prespecificed vector $\x$.

For each example during training, one element of $\x$ was fed into the RNN per time-step.

Similarly for inference time.

This behavior is characteristic of a discriminative network.

Consider: Suppose there were **no** inputs (or more precisely: a very short sequence $\x$ of length $t'$, used to "prime" the RNN).

Instead, let's set the input at time step $t$ to be the output of step $(t-1)$
$$
\x_\tp = \y_{(t-1)}
$$
for $t > t'.

Then the RNN would be self-perpetuating, never exhausting its inputs, and generating new outputs
*conditional* on previous outputs !

This would be a generative form of the RNN.

## Training by teacher forcing

One way to train this RNN is via a supervised task
- given sequence $\x$ up to time $t$: $\x_{(1, \ldots t)}$
- target is $\x_{(t+1)}$

This is just ordinary supervised training with a specially constructed input derived from a single sequence $\x$.  

Of course, we would do this for a training set with many sequences, as usual.

This is similar to a classifier where the class we are trying to predict is the class of the next input
element.

The only "trick" is that, at step $t$, the RNN may output the "wrong" value $\hat{\x} \ne \x_{(t+1)}$.
If we fed the wrong $\hat{\x}$ as the next input to the RNN during training, the RNN would remain
permanently off-track and never learn.

Instead, during *training*, the next input to the RNN 
- is **forced** to be the correct input (**is the input at step $t$ 

$$
\x_{(t+1)} \text{ or is it } \x_{(t+1)}
$$

This type of supervised learning is called *teacher forcing*.

During *inference* time, we feed back as input whatever the generated output is.

### Sampling
We have described a deterministic generation process.

This would be pretty boring, lacking variety
- as well as being problematic for generalization
- we would be encouraging
the RNN to memorize inputs.

In producing the single output, what is really happening is
- our classifier has one logit per
class
- we arbitarily decide to pick the largest.

But we can properly view the (post-softmax) output
- as a probability vector (elements sum to $1$).

Instead of choosing the class with maximum probability
- we can sample from the probability space
defined by this vector
- e.g., if the probability for class $c_1$ is twice as great as that for class
$c_2$, the probability of sampling $c_1$ would be twice as great.  

There is still some chance
that $c_2$is sampled, unlike the deterministic case.

If we do this, the generator can create output sequences different from any training example.

<table>
    <tr>
        <th><center>Training, with Teacher Forcing</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_teacher_forcing_training.jpg" width=800></td>
    </tr>
</table>

<table>
    <tr>
        <th><center>Test time: no forcing</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_teacher_forcing_inference.jpg" width=800"></td>
    </tr>
</table>

## Generating strange things

We haven't specified what each element in sequence $\x$ is.

For text, $x_\tp$ could be either a character or a word, for example.

You'll be surprised how successful an RNN can be when it's task is to consume sequences of characters
and predict the next character.

Although it hasn't been expicitly programmed to generate valid words, punctuation, etc.,
it tends to produce realistic text !

Another interesting fact: these "character RNN's" also learn semantically meaningful constructs
- the need for nested things to match
    - multi-level paranthetical phrases, e.g., "(this is (very important) I think)"
    - opening/closing markup
    - indentation/un-indentation of code blocks

This suggests that the hidden state may be learning to "count" certain concepts.

As we will see in a visualization of the hidden state, and in how LSTM's work, this may in fact be true.

RNN's of this type were quite popular and have been used to generate
- Fake [Shakespeare](http://karpathy.github.io/2015/05/21/rnn-effectiveness/#shakespeare), or fake politician-speak
- Fake code 
- Fake [math textbooks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/#algebraic-geometry-latex)
- [Click bait headline generator](http://clickotron.com/about)


In [3]:
print("Done")

Done
